In [41]:
import scipy.optimize as so
import numpy as np
import h5py as h5
    

In [42]:
def UFid(U,V,Dim):
#    return np.sum(abs(U-V)**2)
    return abs(np.trace(np.matmul(U.conj().T,V))/Dim)**2

In [43]:
def QuantumFourier(Dim): 
    pi=np.arccos(-1.)
    x=np.array(range(Dim))
    return np.exp( 1j*2*pi*np.reshape( np.kron(x,x), [Dim,Dim] )/Dim )/np.sqrt(Dim)

In [44]:
def RandomUnitary(Dim):
    return np.linalg.qr( np.random.randn(Dim,Dim)+1j*np.random.randn(Dim,Dim) )[0]

In [45]:
def UFunction(Phase,N,F,Dim):
    U=np.eye(Dim)
    for k in range(N):
        U= np.exp( 1j*np.append([0] ,Phase[k*(Dim-1):(k+1)*(Dim-1)] ) )[:,np.newaxis]*U
        U=np.matmul(F,U)
    return np.exp(1j*np.append( [0], Phase[N*(Dim-1):(N+1)*Dim-1]))[:,np.newaxis]*U

In [46]:
def PUMIOpt(F,Dim,N,U):
    Seed = np.ones( (N+1)*(Dim-1) )
    TargetOpt = lambda x: 1-UFid( U , UFunction( x , N , F , Dim ) , Dim )
    argsmin = {"method": "L-BFGS-B"}
    Res=so.basinhopping(TargetOpt , Seed,minimizer_kwargs=argsmin,niter=5*10**2,disp=True)
    return Res.fun

In [47]:
def main():
    Repe=100
    Dim=4
    N=Dim
    
    F = QuantumFourier(Dim)
    file=h5.File("FFdataDim=4.h5","r") # Cambiar Nombre archivo 
    UU=file["re"][:,:,:].T+1j*file["im"][:,:,:].T
    
    ind2=file["Index"][:]-1
    MC=len(ind2)
    file.close()
    Data=np.zeros([MC,Repe])
    l=np.where(Data[:,0]<1)
    print(Dim,"-----",MC)
    for j in range(Repe):
        for i in range(MC):
            Data[i,j]=PUMIOpt( F,Dim,N,UU[:,:,ind2[ l[0][i] ]] )
        l2=np.where(Data[l,j]<10**(-10))
        for h in range(np.size(l2[0])):
            Data[l2[0][h],:]=Data[l2[0][h],j]
        l=np.where(Data[:,j]>10**(-10))
        MC=np.size(l[0])
        print(j,"-----",np.size(l[0]))

    file=h5.File("FFdataDim=4py.h5","w") #Cambiar nombre al archivo de salida
    file["infi"]=Data
    file["index1"]=l[0]
    file["index2"]=l2[0]
    file.close()

In [ ]:
infids_comparison = []

for Dim in range(3,11):

    L   = 0
    file=h5.File("data/HaarDistributed_Dim="+str(Dim)+"_Lay=Dim+"+str(L)+".h5","r")
    infids = file['infi'][:]
    file.close()

    infids_mean = np.min(infids,axis=0)
    idx_worst   = np.argmax( infids_mean )
    infid_worst = infids_mean[idx_worst] 

    N=Dim
    F=QuantumFourier(Dim)

    file=h5.File("HaarUnitary.h5","r")
    UU=file['unitarydim={}'.format(Dim)][:,:,:]
    file.close()

    u=UU[idx_worst,:,:]
    infid_BH = PUMIOpt( F,Dim,N,u )

    infids_comparison.append( [infid_worst,infid_BH] )

basinhopping step 0: f 0.000399339
basinhopping step 1: f 0.000160156 trial_f 0.000160156 accepted 1  lowest_f 0.000399339
basinhopping step 2: f 0.000160753 trial_f 0.000160753 accepted 1  lowest_f 0.000399339
basinhopping step 3: f 0.000148484 trial_f 0.000148484 accepted 1  lowest_f 0.000399339
basinhopping step 4: f 7.69642e-05 trial_f 7.69642e-05 accepted 1  lowest_f 0.000399339
basinhopping step 5: f 3.88199e-05 trial_f 3.88199e-05 accepted 1  lowest_f 0.000399339
basinhopping step 6: f 6.69934e-06 trial_f 6.69934e-06 accepted 1  lowest_f 0.000399339
basinhopping step 7: f 4.15362e-06 trial_f 4.15362e-06 accepted 1  lowest_f 0.000399339
basinhopping step 8: f 5.82163e-06 trial_f 5.82163e-06 accepted 1  lowest_f 5.82163e-06
found new global minimum on step 8 with function value 5.82163e-06
basinhopping step 9: f 5.82163e-06 trial_f 7.89806e-06 accepted 0  lowest_f 5.82163e-06
basinhopping step 10: f 5.82163e-06 trial_f 5.26476e-06 accepted 0  lowest_f 5.82163e-06
basinhopping step

In [92]:
infids_comparison

[[0.03516988651982478, 0.03516988652075048],
 [0.01213515036737256, 0.01213515047064928],
 [4.218847493575595e-15, 8.785601135485877e-10],
 [1.0880185641326534e-14, 1.2730787535275567e-09],
 [4.107825191113079e-14, 5.713934436712975e-09],
 [7.09432512735475e-14, 2.07865250567707e-08],
 [1.1368683772161603e-13, 4.4588032155346013e-08],
 [2.949862576429041e-13, 7.539671942335957e-08]]

In [91]:
np.save( 'julia_vs_python', infids_comparison )